In [35]:
from riotwatcher import RiotWatcher
import pandas as pd
from converters.data2frames import game_to_dataframe as g2df
from converters.data2frames import game_teams_to_dataframe as gt2df
from converters.data2frames import champs_to_dataframe as ch2df
from converters.data2files import read_json
from config.constants import API_KEY, SLO_GAMES_DIR, SLO_DATASET_CSV, SLO_MATCHES_FILE_PATH, SCRIMS_POSITIONS_COLS
from config.constants import DATA_DRAGON_URL, DD_RUNES_REFORGED, DD_LANGUAGE, DD_RUNES_REFORGED, STATIC_DATA_DIR
from config.constants import LCK_MATCHES_FILE_PATH, LEAGUES_DATA_DICT, CSV_EXPORT_PATH, EXCEL_EXPORT_PATH, IDS_FILE_PATH, EXPORTS_DIR
import numpy as np
from slds import Slds


In [46]:
slds = Slds(region='EUW1', league='SOLOQ')

In [47]:
rw = slds.rw

In [50]:
tl = rw.match.timeline_by_match(match_id=3567214874, region='EUW1')

In [53]:
fs = tl['frames']

In [63]:
fs[1]['timestamp'] / 60000

1.0038333333333334

In [ ]:
m = rw.match.by_id(match_id=3366842667, region='EUW1')

In [36]:
df2 = pd.read_csv(LEAGUES_DATA_DICT['SOLOQ'][CSV_EXPORT_PATH], index_col=0, encoding='ISO-8859-1')

C:\Users\david\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
df2['lanerole'] = df2['lane'] + '_'  + df2['role']

In [43]:
df2.lanerole.value_counts()

JUNGLE_NONE           7379
BOTTOM_DUO_SUPPORT    6320
BOTTOM_DUO_CARRY      6264
TOP_SOLO              6157
MIDDLE_SOLO           6013
NONE_DUO_SUPPORT      3000
MIDDLE_DUO             769
NONE_DUO               639
TOP_DUO                569
BOTTOM_DUO             350
BOTTOM_SOLO            347
MIDDLE_DUO_SUPPORT     314
MIDDLE_DUO_CARRY       116
TOP_DUO_SUPPORT        104
TOP_DUO_CARRY           76
MIDDLE_NONE              3
NONE_DUO_CARRY           1
Name: lanerole, dtype: int64

In [45]:
df2[df2.lanerole == 'NONE_DUO_SUPPORT'].champ_name

150            Shen
151           Corki
153          Thresh
154       Jarvan IV
155        Tristana
156           Janna
157            Ryze
158         Hecarim
159       Gangplank
260          Maokai
261            Ryze
262            Nunu
263           Xayah
264           Rakan
265      Tahm Kench
266           Galio
267           Shaco
268        Tristana
269          Singed
270           Riven
271           Varus
272            Ahri
273             Zac
274           Janna
275         Skarner
276         Caitlyn
277            Lulu
278        Vladimir
279          Irelia
280      Cassiopeia
            ...    
38227         Varus
38228          Kayn
38229      Kassadin
38230         Karma
38321         Braum
38322         Jayce
38323       Kha'Zix
38324         Sivir
38325        Anivia
38326     Gangplank
38327         Janna
38328       Evelynn
38329          Ryze
38391         Vayne
38392           Zoe
38393          Lulu
38394       Skarner
38395         Fiora
38396          Ekko


In [ ]:
accs = [23787352, 227994782, 203396130, 231898666, 22654093, 36549969, 33328489, 218694983, 35306044, 36623216, 26609836]

In [ ]:
accs

In [ ]:
pd.DataFrame({'game_id': accs})

In [ ]:
from itertools import chain

In [ ]:
matches = list(chain.from_iterable([rw.match.matchlist_by_account(account_id=acc, end_index=20, region=slds.region, queue=420)['matches'] for acc in accs]))

In [ ]:
gids = list(set([m['gameId'] for m in mls]))

In [ ]:
df1 = pd.read_csv(LEAGUES_DATA_DICT['LCK'][IDS_FILE_PATH], dtype=LEAGUES_DATA_DICT['LCK']['dtypes'])

In [ ]:
list(df1.game_id.map(str) + df1.tournament + df1.hash)

In [ ]:
def something(asd, **kwargs):
    print(kwargs['b'])

In [ ]:
something('asd', b='shit')

In [ ]:
pids = {"participantId":1,"player":{"summonerName":"SKT Untara","profileIcon":4}},{"participantId":2,"player":{"summonerName":"SKT Blossom","profileIcon":7}},{"participantId":3,"player":{"summonerName":"SKT Faker","profileIcon":7}},{"participantId":4,"player":{"summonerName":"SKT Bang","profileIcon":20}},{"participantId":5,"player":{"summonerName":"SKT Wolf","profileIcon":7}},{"participantId":6,"player":{"summonerName":"KZ Khan","profileIcon":6}},{"participantId":7,"player":{"summonerName":"KZ Peanut","profileIcon":6}},{"participantId":8,"player":{"summonerName":"KZ Bdd","profileIcon":6}},{"participantId":9,"player":{"summonerName":"KZ PraY","profileIcon":6}},{"participantId":10,"player":{"summonerName":"KZ GorillA","profileIcon":6}}

In [ ]:
for x in pids:
    print(x)